### Adjust Word Level

In [164]:
#!pip install xlsxwriter

In [39]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


#### While Loop

In [34]:
lang_folder = "Turkish"  # Primary language
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian

In [35]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
164294,buldugumuzda,43
164295,boşandıklarını,42
164296,endişenlenme,42
164297,uzaylıymışım,42


In [36]:
#df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
df_twogram_sent

,twogram,frequency
0,teşekkür ederim,244149
1,öyle mi,209900
2,ne oldu,195799
3,aman tanrım,189521
4,özür dilerim,153784
...,...,...
1036515,güzeldi tommy,3
1036516,durumu tuhaflaştırma,3
1036517,güzeldi canım,3
1036518,güzeldi daniel,3


In [37]:
lang_pair_list = glob.glob(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()}_And_{lang_pair.lower().capitalize()}*_All.xlsx")
lang_pair_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/Turkish/Turkish_And_Dutch (nld)_Etymologeek_Result_All.xlsx']

In [38]:
df_pair = pd.read_excel(f"{lang_pair_list[0]}")
df_pair

,search_word,dict_entry_main,language_main,definition_main,dict_entry,language,definition
0,şans,şans,Turkish,(Noun) Fortune. Luck.Fortune. Luck.,dice,Dutch (nld),NaN
1,loca,loca,Turkish,(Noun) (masonic) lodge. Box (in theaters).(mas...,laubja,Dutch (nld),NaN


In [8]:
def remove_repetition(word_group):
    words = word_tokenize(word_group)
    word_unique = set(words)
    if len(word_unique) == 1:
        return "repetitive_word_group"
    else:
        return word_group

In [9]:
def word_count_result(df,column_list): # df is dataframe, bigger_than is integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [10]:
def word_count_bool(df, word_thresh_num, column_list): # df is a dataframe, bigger_than is an integer
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list)
    df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"count"}, inplace=True)
    
    return  df_word_count["count"][df_word_count.loc[:,"count"] > word_thresh_num].any()  

#### Condition 1

In [11]:
# while loop code block word and twogram pair
twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
word_thresh_num = 2

word_start = 0  # 0
word_end = 7  # 10
step_num = word_end  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)

    # output
    df_output_result = pd.concat([df_word, df_lang_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word 
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert twogram pair
        try:
            var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
            freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, word_thresh_num, [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        a += 1
                        var2 = df_output_result.loc[a,f"twogram_pair_{lang_pair.lower()}"]
                        freq_var2 = df_output_result.iloc[a,3]  # twogram_pair frequency
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

    df_lesson_word_count = word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Condition 2

In [12]:
part_result_file = glob.glob(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result*.xlsx")
part_result_file

['Turkish_Portuguese_28_Word_Step_7_Result1.xlsx',
 'Turkish_Portuguese_28_Word_Step_7_Result2.xlsx',
 'Turkish_Portuguese_28_Word_Step_7_Result3.xlsx',
 'Turkish_Portuguese_28_Word_Step_7_Result4.xlsx']

In [13]:
# Read previous part result
df_part_all = pd.DataFrame()
for i in part_result_file:
    df_var = pd.read_excel(f"{i}")
    df_part_all = pd.concat([df_part_all,df_var], axis=0)
df_part_twogram_pair = df_part_all.loc[:,[f"twogram_pair_{lang_pair.lower()}"]]
df_part_twogram_pair.reset_index(drop=True, inplace=True)
set_part_twogram_pair = set(df_part_twogram_pair[f"twogram_pair_{lang_pair.lower()}"])

# while loop code block
twogram_num = 100  # 2*step_num minimum: for each word takes two twogram
twogram_pair_num = 100  # 2*step_num minimum: for each word takes two twogram pair
word_thresh_num = 7

word_start = 0  # 0
word_end = 28  # 10
step_num = word_end  # 10
word_limit = 28  # 200
part_num = 1
while word_end <= word_limit:
    df_word = df_word_all.iloc[word_start:word_end,]
    df_word.reset_index(drop=True, inplace=True)

    # language pair twogram
    ngram_list = []
    for i in df_pair["dict_entry_main"]:
        for j in word_tokenize(i):
            for k in df_word["word"]:
                twogram_1_2 = f"{j} {k}"
                ngram_list.append(twogram_1_2)
                twogram_2_1 = f"{k} {j}"
                ngram_list.append(twogram_2_1)
    df_pair_ngram = pd.DataFrame(ngram_list)
    df_pair_ngram.rename(columns={0:"twogram"}, inplace=True)
    df_pair_ngram.iloc[:,0] = df_pair_ngram.iloc[:,0].apply(lambda x: remove_repetition(x))
    df_pair_ngram.drop_duplicates(inplace=True)
    df_pair_ngram.reset_index(drop=True, inplace=True)
    df_lang_pair_twogram = pd.merge(df_twogram_sent, df_pair_ngram, how="inner", on="twogram")
    df_lang_pair_twogram.rename(columns={"twogram":f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_lang_pair_twogram.drop_duplicates(inplace=True)
    #df_lang_pair_twogram = df_lang_pair_twogram.head(100)
    set_lang_pair_twogram = set(df_lang_pair_twogram[f"twogram_pair_{lang_pair.lower()}"])
    df_set_result = pd.DataFrame(set_lang_pair_twogram.difference(set_part_twogram_pair))
    df_set_result.rename(columns={0:f"twogram_pair_{lang_pair.lower()}"}, inplace=True)
    df_set_pair_twogram = pd.merge(df_lang_pair_twogram, df_set_result, how="inner", on=f"twogram_pair_{lang_pair.lower()}")

    # twogram
    word_list = df_word["word"].values.tolist()
    data_kind = "twogram"
    twogram_list  = df_twogram_sent.iloc[:,0].values.tolist()
    
    resultlist2 = []

    manager = multiprocessing.Manager()
    resultlist2 = manager.list()
    
    def word_in_wordgroup2(list_var2):
        mergelist = []
        try:
            word = list_var2.split()
        except:
            pass
        var1 = range(len(word))
        for j in var1:
            if word[j] in word_list:
                mergelist.append(word[j])
                if len(mergelist) == len(word):
                        resultlist2.append(list_var2)
                            
    if __name__ == '__main__':
        # with Pool(16) as p:
        with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
            p.map(word_in_wordgroup2, twogram_list) # string_word liste 

    result_list2 = list(resultlist2)
    df_result2 = pd.DataFrame(result_list2)
    df_result2 = df_result2.rename(columns = {0: f"{data_kind}"})
    df_result2.iloc[:,0] = df_result2.iloc[:,0].apply(lambda x: remove_repetition(x)) # **
    df_merge2 = pd.merge(df_result2, df_twogram_sent, how="inner", on=f"{data_kind}")
    df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
    df_merge_result2.drop_duplicates(inplace=True)
    df_merge_result2.reset_index(drop=True, inplace=True)
    df_twogram_result = df_merge_result2
    #df_twogram_result = df_twogram_result.head(100)

    # output
    df_output_result = pd.concat([df_word, df_twogram_result, df_set_pair_twogram], axis=1)

    df_lesson_result = pd.DataFrame(columns=["word","freq_word","twogram","freq_twogram",f"twogram_pair_{lang_pair.lower()}",f"freq_twogram_pair_{lang_pair.lower()}"])
    a = 0
    b = 0

    for i in range(0,110):
        # Insert words and their count 
        try:
            word = df_output_result.iloc[i,0]  # word
            freq_word = df_output_result.iloc[i,1]  # word freq
            df_lesson_result.loc[i,"word"] = word
            df_lesson_result.loc[i,"freq_word"] = freq_word
        except:
            pass
         
        # Insert n grams
        try:
            var1 = df_output_result.iloc[a,2]
            freq_var1 = df_output_result.iloc[a,3]
            if (len(df_lesson_result["twogram"]) < twogram_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,"twogram"] = var1
                df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-5), ["twogram"]): # word count result                
                        a += 1
                        var1 = df_output_result.iloc[a,2]
                        freq_var1 = df_output_result.iloc[a,3]
                        df_lesson_result.loc[i,"twogram"] = var1
                        df_lesson_result.loc[i,"freq_twogram"] = freq_var1
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,"twogram"] = np.nan
                    df_lesson_result.loc[i,"freq_twogram"] = np.nan
            else:
                pass
        except:
            pass
        a += 1

        try:
            var2 = df_output_result.iloc[b,4]
            freq_var2 = df_output_result.iloc[b,5]
            if (len(df_lesson_result[f"twogram_pair_{lang_pair.lower()}"]) < twogram_pair_num): # and (not(word_count(df_lesson_result, word_thresh_num))):
                df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                try:
                    while word_count_bool(df_lesson_result, (word_thresh_num-5), [f"twogram_pair_{lang_pair.lower()}"]): # word count result                
                        b += 1
                        var2 = df_output_result.iloc[b,4]
                        freq_var2 = df_output_result.iloc[b,5]
                        df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = var2
                        df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = freq_var2
                    else:
                        pass
                except:
                    df_lesson_result.loc[i,f"twogram_pair_{lang_pair.lower()}"] = np.nan
                    df_lesson_result.loc[i,f"freq_twogram_pair_{lang_pair.lower()}"] = np.nan
            else:
                pass
        except:
            pass
        b += 1

    df_lesson_word_count = word_count_result(df_lesson_result, ["twogram",f"twogram_pair_{lang_pair.lower()}"])
    df_lesson_result = pd.merge(df_lesson_result, df_lesson_word_count, how="left", on="word")
    df_lesson_result = df_lesson_result.drop_duplicates()
    df_lesson_result.to_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_{step_num}_Result{part_num}.xlsx", index=False)

    word_start += step_num
    word_end += step_num
    part_num += 1 

#### Check Result

In [14]:
df_lesson_result

,word,freq_word,twogram,freq_twogram,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735.0,ne var,62532,harita için,15.0,3.0
1,bu,11062659.0,ben de,59972,bu harita,12.0,3.0
2,ne,8025880.0,değil mi,58386,ve sandviç,7.0,2.0
3,ve,7766036.0,ben mi,33652,bir mango,5.0,3.0
4,için,5484109.0,çok iyi,33627,sandviç var,3.0,2.0
5,mi,5362714.0,ne için,31857,NaN,NaN,2.0
6,ben,4908913.0,hayır değil,18740,NaN,NaN,2.0
7,de,4880315.0,hayır mı,15769,NaN,NaN,2.0
8,çok,4852169.0,bu kadar,15745,NaN,NaN,2.0
9,ama,4661966.0,evet var,11138,NaN,NaN,2.0


In [1]:
#word_count_result(df_lesson_result, ["twogram"])

In [2]:
#word_count_result(df_lesson_result, [f"twogram_pair_{lang_pair.lower()}"])

#### Read Output File And Multi Write

In [17]:
df_part_11 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result1.xlsx")
df_part_11

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735,bir harita,247.0,2.0
1,bu,11062659,sandviç mi,111.0,1.0
2,ne,8025880,bir sandviç,43.0,NaN
3,ve,7766036,harita bu,17.0,NaN
4,için,5484109,köri mi,13.0,NaN
5,mi,5362714,NaN,NaN,2.0
6,ben,4908913,NaN,NaN,NaN


In [18]:
df_part_12 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result2.xlsx")
df_part_12

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,de,4880315,harita mı,166.0,1.0
1,çok,4852169,evet harita,9.0,NaN
2,ama,4661966,sandviç değil,4.0,NaN
3,var,4389551,sandviç de,3.0,NaN
4,evet,4324786,NaN,NaN,1.0
5,mı,4001316,NaN,NaN,1.0
6,değil,3883885,NaN,NaN,1.0


In [19]:
df_part_13 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result3.xlsx")
df_part_13

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,da,3610161,hayır mango,10.0,NaN
1,şey,3602024,NaN,NaN,NaN
2,hayır,3406992,NaN,NaN,1.0
3,daha,3317577,NaN,NaN,NaN
4,sen,3283654,NaN,NaN,NaN
5,kadar,2697900,NaN,NaN,NaN
6,bana,2659182,NaN,NaN,NaN


In [20]:
df_part_14 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_7_Result4.xlsx")
df_part_14

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,yok,2491685,harita yok,25.0,1.0
1,onu,2486889,NaN,NaN,NaN
2,seni,2454988,NaN,NaN,NaN
3,beni,2446696,NaN,NaN,NaN
4,bunu,2445337,NaN,NaN,NaN
5,gibi,2427957,NaN,NaN,NaN
6,iyi,2383224,NaN,NaN,NaN


In [21]:
df_part_21 = pd.read_excel(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Step_28_Result1.xlsx")
df_part_21

,word,freq_word,twogram,freq_twogram,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735,ne var,62532.0,harita için,15.0,3.0
1,bu,11062659,ben de,59972.0,bu harita,12.0,3.0
2,ne,8025880,değil mi,58386.0,ve sandviç,7.0,2.0
3,ve,7766036,ben mi,33652.0,bir mango,5.0,3.0
4,için,5484109,çok iyi,33627.0,sandviç var,3.0,2.0
5,mi,5362714,ne için,31857.0,NaN,NaN,2.0
6,ben,4908913,hayır değil,18740.0,NaN,NaN,2.0
7,de,4880315,hayır mı,15769.0,NaN,NaN,2.0
8,çok,4852169,bu kadar,15745.0,NaN,NaN,2.0
9,ama,4661966,evet var,11138.0,NaN,NaN,2.0


In [22]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Result_With_Frequency.xlsx", engine='xlsxwriter')

In [23]:
df_part_11.to_excel(writer, sheet_name='Word_Part11', index=False)
df_part_12.to_excel(writer, sheet_name='Word_Part12', index=False)
df_part_13.to_excel(writer, sheet_name='Word_Part13', index=False)
df_part_14.to_excel(writer, sheet_name='Word_Part14', index=False)
df_part_21.to_excel(writer, sheet_name='Word_Part21', index=False)

In [24]:
writer.save()

#### Output File Word Order

In [25]:
df_part_11

,word,freq_word,twogram_pair_portuguese,freq_twogram_pair_portuguese,word_count
0,bir,18835735,bir harita,247.0,2.0
1,bu,11062659,sandviç mi,111.0,1.0
2,ne,8025880,bir sandviç,43.0,NaN
3,ve,7766036,harita bu,17.0,NaN
4,için,5484109,köri mi,13.0,NaN
5,mi,5362714,NaN,NaN,2.0
6,ben,4908913,NaN,NaN,NaN


In [26]:
def word_in_twogram(df, list_column, target_column):

    '''word_in_twogram(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_twogram.insert(0,"word",i)
        df_word_result = pd.concat([df_word_result,word_in_twogram], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [27]:
word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}") 

,word,twogram_pair_portuguese
0,bir,bir harita
1,bir,bir sandviç
2,bu,harita bu
3,mi,sandviç mi
4,mi,köri mi


In [28]:
df_word_order_11 = word_in_twogram(df_part_11, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_12 = word_in_twogram(df_part_12, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_13 = word_in_twogram(df_part_13, "word", f"twogram_pair_{lang_pair.lower()}") 
df_word_order_14 = word_in_twogram(df_part_14, "word", f"twogram_pair_{lang_pair.lower()}")
df_word_order_21 = word_in_twogram(df_part_21, "word", f"twogram") 
df_word_order_212 = word_in_twogram(df_part_21, "word", f"twogram_pair_{lang_pair.lower()}")  

In [29]:
df_word_order_join_11 = df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_12 = df_word_order_12.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_13 = df_word_order_13.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_14 = df_word_order_14.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_211 = df_word_order_21.groupby(["word"])["twogram"].apply(", ".join).reset_index()
df_word_order_join_212 = df_word_order_212.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].apply(", ".join).reset_index()
df_word_order_join_21 = pd.merge(df_word_order_join_211, df_word_order_join_212, how="outer", on="word")

In [30]:
df_word_order_join_21

,word,twogram,twogram_pair_portuguese
0,ama,"evet ama, ama yok",NaN
1,bana,"bana mı, bana da",NaN
2,ben,"ben de, ben mi",NaN
3,beni,"seni beni, beni seni",NaN
4,bir,"bir daha, bir şey",bir mango
5,bu,"bu kadar, bu iyi",bu harita
6,bunu,"bunu da, ve bunu",NaN
7,da,"bana da, bunu da",NaN
8,daha,"bir daha, daha çok",NaN
9,de,"ben de, sen de",NaN


In [31]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_{word_limit}_Word_Join_Result_Without_Frequency.xlsx", engine='xlsxwriter')

In [32]:
df_word_order_join_11.to_excel(writer2, sheet_name='Word_Part11', index=False)
df_word_order_join_12.to_excel(writer2, sheet_name='Word_Part12', index=False)
df_word_order_join_13.to_excel(writer2, sheet_name='Word_Part13', index=False)
df_word_order_join_14.to_excel(writer2, sheet_name='Word_Part14', index=False)
df_word_order_join_21.to_excel(writer2, sheet_name='Word_Part21', index=False)

In [33]:
writer2.save()